In [1]:
import sys
sys.path.insert(0, "../../")

import inspect

In [2]:
from collections import OrderedDict    # For recording the model specification 

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations
import statsmodels.tools.numdiff as numdiff       # For numeric hessian
import scipy.linalg                    # For matrix inversion

import pylogit as pl                   # For choice model estimation
from pylogit import nested_logit as nl # For nested logit convenience funcs

In [3]:
inspect.getfile(pl)

'/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/pylogit/__init__.py'

In [47]:
trips = pd.read_csv('/home/emma/activitysynth/activitysynth/data/HWtrips_042519.csv')

In [48]:
trips['DRIVE'] = 1
trips['SHARED'] = 1
trips['WALK_TRANSIT'] = 1
trips['DRIVE_TRANSIT'] = 1
trips['BIKE'] = 1
trips['WALK'] = 1

In [50]:
trips["custom_id"] = np.arange(trips.shape[0], dtype=int) + 1

In [51]:
pd.set_option('display.max_columns', None)
trips.head()

,Unnamed: 0,HHPER,index,Unnamed: 0.1,HHPERTRIP,origin,home_dwell,home_ST,HW_trip_ST,HW_trip_ET,HW_TT,Hzone,Wzone,Hxcord,Hycord,Wxcord,Wycord,HW_mode,work_dwell,work_ST,WH_trip_ST,WH_trip_ET,WH_TT,WH_mode,modeHW,TOD,dwell_work,SAMPN,PERNO,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,hours,EDUCA,INDUS,HHVEH,OWN,INCOM,HHSIZ,minority,black,native,asian,PI,hh_inc_less10k,hh_inc_10kless25k,hh_inc_25kless35k,hh_inc_35kless50k,hh_inc_50kless75k,hh_inc_100kless150k,hh_inc_150kless200k,hh_inc_200kless250k,hh_inc_less50k,hh_inc_less75k,hh_inc_75kless100k,hh_inc_150kplus,hh_inc_150kless250k,hh_inc_250kplus,lessGED,GED,somebach,Assoc,Bach,lessGED_GED,lessbach,age_16less25,age_25less40,age_40less50,age_50less60,female,tenure_2,noveh,1veh,2veh,3plusveh,hh_size_1per,hh_size_2per,hh_size_3per,hh_size_4plusper,sector_constr,sector_mfg,sector_retail,sector_transport,info,finance,scitech,sector_edu_serv,sector_healthcare,sector_oth_serv,sector_gov,TOD_3to6,TOD_6to9,TOD_9to1530,TOD_1530to1830,TOD_1830up,TOD_9up,TOD_1530up,from_zone_id,to_zone_id,gen_tt_CAR_AM,gen_tt_BIKE_AM,gen_tt_DRIVE_TRANSIT_AM,gen_tt_WALK_AM,gen_tt_WALK_TRANSIT_AM,gen_tt_CAR_PM,gen_tt_BIKE_PM,gen_tt_DRIVE_TRANSIT_PM,gen_tt_WALK_PM,gen_tt_WALK_TRANSIT_PM,gen_tt_CAR_OffPeak,gen_tt_BIKE_OffPeak,gen_tt_DRIVE_TRANSIT_OffPeak,gen_tt_WALK_OffPeak,gen_tt_WALK_TRANSIT_OffPeak,total_jobs_gen_tt_CAR_AM_30,total_jobs_gen_tt_WALK_TRANSIT_AM_30,total_jobs_gen_tt_DRIVE_TRANSIT_AM_30,total_jobs_gen_tt_BIKE_AM_30,total_jobs_gen_tt_WALK_AM_30,sum_residential_units_gen_tt_CAR_AM_30,sum_residential_units_gen_tt_WALK_TRANSIT_AM_30,sum_residential_units_gen_tt_DRIVE_TRANSIT_AM_30,sum_residential_units_gen_tt_BIKE_AM_30,sum_residential_units_gen_tt_WALK_AM_30,sum_persons_gen_tt_CAR_AM_30,sum_persons_gen_tt_WALK_TRANSIT_AM_30,sum_persons_gen_tt_DRIVE_TRANSIT_AM_30,sum_persons_gen_tt_BIKE_AM_30,sum_persons_gen_tt_WALK_AM_30,avg_income_gen_tt_CAR_AM_30,avg_income_gen_tt_WALK_TRANSIT_AM_30,avg_income_gen_tt_DRIVE_TRANSIT_AM_30,avg_income_gen_tt_BIKE_AM_30,avg_income_gen_tt_WALK_AM_30,gen_tt_CAR,gen_tt_BIKE,gen_tt_WALK,gen_tt_WALK_TRANSIT,gen_tt_DRIVE_TRANSIT,DRIVE,SHARED,WALK_TRANSIT,DRIVE_TRANSIT,BIKE,WALK,custom_id
0,3992,30202193,9223,9547,3.020219e+09,home,14.583333,7.166667,21.750000,22.000000,0.250000,525.0,523.0,-121.896879,37.280818,-121.918797,37.264669,drive_alone,9.000000,22.000000,7.000000,7.166667,0.166667,drive_alone,0,4,4,3020219,3,1.0,21.0,0,1.0,NaN,NaN,NaN,30.0,2.0,44.0,1,1.0,6.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,525,523,3.509251,13.350405,5.868917,58.325964,8.841972,4.349597,13.422655,6.335453,52.656144,8.967193,3.276973,13.390643,5.991765,51.472567,8.924065,383050.0,186953.0,329860.0,75337.0,826.0,378239.0,233326.0,387325.0,106794.0,3272.0,998095.0,579609.0,1026804.0,253673.0,8064.0,282587.520824,280611.766021,284329.195532,279697.715188,336615.636364,3.276973,13.390643,51.472567,8.924065,5.991765,1,1,1,1,1,1,1
1,3203,25893411,7394,7663,2.589341e+09,home,13.933333,17.783333,7.716667,8.766667,1.050000,183.0,69.0,-122.496138,37.736289,-122.450262,37.776308,drive_alone,8.033333,8.766667,16.800000,17.783333,0.983333,drive_alone,0,1,3,2589341,1,2.0,37.0,0,4.0,NaN,NaN,NaN,40.0,6.0,52.0,1,1.0,8.0,5,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,183,69,9.487323,36.093055,15.866721,157.685268,23.904427,11.759213,36.288384,17.128008,142.356813,24.242964,14.166667,36.201839,16.198843,139.156989,24.126367,406461.0,104443.0,374588.0,31419.0,1541.0,436772.0,261837.0,420902.0,118796.0,10802.0,915147.0,599577.0,895780.0,273038.0,27471.0,227631.760137,229196.431064,227579.411547,235577.940921,220483.941379,9.487323,36.093055,157.685268,23.904427,15.866721,1,1,1,1,1,1,2
2,3155,25771411,7301,7560,2.577141e+09,home,13.433333,18.483333,7.916667,8.116667,0.200000,358.0,345.0,-122.133233,37.441851,-122.193415,37.425610,drive_alone,10.116667,8.116667,18.233

In [52]:
# Create the list of individual specific variables
ind_variables = ['TOD_3to6', 'sector_mfg','finance','scitech','sector_edu_serv','sector_healthcare','sector_gov',
                'age_16less25','age_25less40','age_50less60','female','hh_inc_less75k',
                 'lessGED_GED','somebach','Assoc','lessbach',
                 'noveh','hh_size_3per','hh_size_4plusper','tenure_2',
                'total_jobs_gen_tt_CAR_AM_30','total_jobs_gen_tt_WALK_AM_30',
                'sum_residential_units_gen_tt_CAR_AM_30','sum_persons_gen_tt_CAR_AM_30']

In [53]:
ind_variables

['TOD_3to6',
 'sector_mfg',
 'finance',
 'scitech',
 'sector_edu_serv',
 'sector_healthcare',
 'sector_gov',
 'age_16less25',
 'age_25less40',
 'age_50less60',
 'female',
 'hh_inc_less75k',
 'lessGED_GED',
 'somebach',
 'Assoc',
 'lessbach',
 'noveh',
 'hh_size_3per',
 'hh_size_4plusper',
 'tenure_2',
 'total_jobs_gen_tt_CAR_AM_30',
 'total_jobs_gen_tt_WALK_AM_30',
 'sum_residential_units_gen_tt_CAR_AM_30',
 'sum_persons_gen_tt_CAR_AM_30']

In [54]:
# alt_varying_variables = ('gen_tt_CAR','gen_tt_WALK_TRANSIT','gen_tt_DRIVE_TRANSIT','gen_TT_BIKE','gen_tt_WALK')

In [55]:
# Specify the variables that vary across individuals **AND** 
# across some or all alternatives
alt_varying_variables = {u'travel_time': dict([(0, 'gen_tt_CAR'),
                                               (1, 'gen_tt_CAR'),
                                               (2, 'gen_tt_WALK_TRANSIT'),
                                               (3, 'gen_tt_DRIVE_TRANSIT'),
                                               (5, 'gen_tt_BIKE'),
                                               (6, 'gen_tt_WALK')])}

In [56]:
# Specify the availability variables
availability_variables = {0:'DRIVE',
                          1:'SHARED',
                          2:'WALK_TRANSIT',
                          3:'DRIVE_TRANSIT',
                          5: 'BIKE',
                          6: 'WALK'}

In [58]:
# Determine the columns that will denote the
# new column of alternative ids, and the columns
# that denote the custom observation ids and the 
# choice column
new_alt_id = "mode_id"
obs_id_column = "custom_id"
choice_column = "modeHW"

In [59]:
# Perform the desired conversion
trips1 = pl.convert_wide_to_long(trips, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=new_alt_id)

# Look at the first 9 rows of the long-format dataframe
trips1.head(9).T

/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/pylogit/choice_tools.py:453: UserWarning: Note, there are 150 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 37 variables.
  msg_2 + msg_3.format(num_vars_accounted_for))


,0,1,2,3,4,5,6,7,8
custom_id,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
mode_id,0.000000,1.000000,2.000000,3.000000,5.000000,6.000000,0.000000,1.000000,2.000000
modeHW,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
TOD_3to6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_mfg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
finance,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
scitech,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_edu_serv,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_healthcare,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_gov,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
trips1.head(9).T

,0,1,2,3,4,5,6,7,8
custom_id,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
mode_id,0.000000,1.000000,2.000000,3.000000,5.000000,6.000000,0.000000,1.000000,2.000000
modeHW,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
TOD_3to6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_mfg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
finance,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
scitech,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_edu_serv,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_healthcare,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_gov,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [61]:
trips1["mode_id"] = pd.to_numeric(trips1["mode_id"])

In [94]:
# Specify the nesting values
nest_membership = OrderedDict()
nest_membership["car"] = [0,1]
nest_membership["transit"] = [2,3]
nest_membership["active"] = [5,6]

In [95]:
# Create the model's specification dictionary and variable names dictionary
# NOTE: - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         or lists of lists. Within a list, or within the inner-most
#         list should be the alternative ID's of the alternative whose
#         utility specification the explanatory variable is entering.

example_specification = OrderedDict()
example_names = OrderedDict()

# Note that 1 is the id for the Train and 3 is the id for the Car.
# The next two lines are placing alternative specific constants in
# the utility equations for the Train and for the Car. The order
# in which these variables are placed is chosen so the summary
# dataframe which is returned will match that shown in the HTML
# file of the python biogeme example.
example_specification["intercept"] = [0,3,5]
example_names["intercept"] = ['ASC Car', 'ASC Drive_Transit', 'ASC Bike']

# Note that the names used below are simply for consistency with
# the coefficient names given in the Python Biogeme example.
# example_specification["travel_cost_hundredth"] = [[1, 2, 3]]
# example_names["travel_cost_hundredth"] = ['B_COST']

example_specification["travel_time"] = [[0,1],[2,3],5,6]
example_names["travel_time"] = ['Car TT','Transit TT','Bike TT','Walk TT']

example_specification["total_jobs_gen_tt_CAR_AM_30"] = [0]
example_names["total_jobs_gen_tt_CAR_AM_30"] = ['Total Jobs_30_CAR']

example_specification["total_jobs_gen_tt_WALK_AM_30"] = [6]
example_names["total_jobs_gen_tt_WALK_AM_30"] = ['Total Jobs_30_WALK']

example_specification["sum_residential_units_gen_tt_CAR_AM_30"] = [0,1]
example_names["sum_residential_units_gen_tt_CAR_AM_30"] = ['Sum Res Units_30_CAR','Sum Res Units_30_shared']

example_specification["sum_persons_gen_tt_CAR_AM_30"] = [0,1]
example_names["sum_persons_gen_tt_CAR_AM_30"] = ['Sum Persons_30_CAR','Sum Persons_30_shared']


example_specification["TOD_3to6"] = [3]
example_names["TOD_3to6"] = ['TOD_3to6_DrTran']

example_specification["sector_mfg"] = [3]
example_names["sector_mfg"] = ['sector_mfg_DrTran']

example_specification["finance"] = [2,3]
example_names["finance"] = ['finance_WTran','finance_DrTran']

example_specification["scitech"] = [3]
example_names["scitech"] = ['scitech_DrTran']

example_specification["sector_edu_serv"] = [5,6]
example_names["sector_edu_serv"] = ['sector_edu_serv_bike','sector_edu_serv_walk']

example_specification["sector_healthcare"] = [3]
example_names["sector_healthcare"] = ['sector_healthcare_DrTran']

example_specification["sector_gov"] = [2]
example_names["sector_gov"] = ['sector_gov_WTran']

example_specification["age_16less25"] = [1]
example_names["age_16less25"] = ['age_16less25_shared']  

example_specification["age_25less40"] = [2,5]
example_names["age_25less40"] = ['age_25less40_WTran','age_25less40_bike'] 

example_specification["age_50less60"] = [1]
example_names["age_50less60"] = ['age_50less60_shared']  

example_specification["female"] = [1,5]
example_names["female"] = ['female_shared','female_bike'] 

example_specification["hh_inc_less75k"] = [1,2]
example_names["hh_inc_less75k"] = ['hh_inc_less75k_shared','hh_inc_less75k_WTran'] 

example_specification["lessGED_GED"] = [2]
example_names["lessGED_GED"] = ['lessGED_GED_WTran']

example_specification["somebach"] = [2]
example_names["somebach"] = ['somebach_WTran']

example_specification["Assoc"] = [2,6]
example_names["Assoc"] = ['Assoc_WTran','Assoc_walk']

example_specification["lessbach"] = [5]
example_names["lessbach"] = ['lessbach_bike']

example_specification["noveh"] = [1,2,5,6]
example_names["noveh"] = ['noveh Shared', 'noveh Walk_Transit', 'noveh Bike', 'noveh Walk']
    
example_specification["hh_size_3per"] = [2]
example_names["hh_size_3per"] = ['hh_size_3per_WTran']

example_specification["hh_size_4plusper"] = [2]
example_names["hh_size_4plusper"] = ['hh_size_4plusper_WTran']
    
example_specification["tenure_2"] = [2,6]
example_names["tenure_2"] = ['rent_WTran','rent_walk']

In [81]:
# Define a function that calculates the "logit" transformation of values
# between 0.0 and 1.0.
def logit(x):
    """
    Parameters
    ----------
    x : int, float, or 1D ndarray.
        If an array, all elements should be ints or floats. All
        elements should be between zero and one, exclusive of 1.0.

    Returns
    -------
    The logit of x:  `np.log(x / (1.0 - x))`.
    """
    return np.log(x/(1.0 - x))

In [97]:
# Provide the module with the needed input arguments to create
# an instance of the MNL model class
example_nested = pl.create_choice_model(data=trips1,
                                        alt_id_col=new_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=example_specification,
                                        model_type="Nested Logit",
                                        names=example_names,
                                        nest_spec=nest_membership)

# # Specify the initial nesting parameter values
# # Note: This should be in terms of the reparameterized values used
# # by PyLogit.

# # Note: The '40' corresponds to scale parameter that is numerically
# # indistinguishable from 1.0

# # Note: 2.05 is the scale parameter that is estimated by PythonBiogeme
# # so we invert it, then take the logit of this inverse to get the
# # corresponding starting value to be used by PyLogit.
# # Note the first value corresponds to the first nest in 'nest_spec'
# # and the second value corresponds to the second nest in 'nest_spec'.
# init_nests = np.array([40, logit(2.05**-1)])

# # Specify the initial index coefficients used by PythonBiogeme
# init_coefs = np.array([-0.167, -0.512, -0.899, -0.857])

# # Create a single array of the initial values
# init_values = np.concatenate((init_nests, init_coefs), axis=0)

# Start the model estimation from the pythonbiogeme initial values
# Note that the first value, in the initial values, is constrained
# to remain constant through the estimation process. This is because
# the first nest in nest_spec is a 'degenerate' nest with only one
# alternative, and the nest parameter of degenerate nests is not
# identified.
example_nested.fit_mle(np.zeros(46))

Log-likelihood at zero: -6,572.1737
Initial Log-likelihood: -6,572.1737


/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/scipy/optimize/_minimize.py:506: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 2.44 minutes.
Final log-likelihood: -2,624.3745


/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/pylogit/base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),
/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [98]:
# Look at the estimated coefficients and goodness-of-fit statistics
example_nested.get_statsmodels_summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Nested Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 modeHW   No. Observations:                3,668
Model:             Nested Logit Model   Df Residuals:                    3,622
Method:                           MLE   Df Model:                           46
Date:                Thu, 25 Apr 2019   Pseudo R-squ.:                   0.601
Time:                        18:17:13   Pseudo R-bar-squ.:               0.594
AIC:                        5,340.749   Log-Likelihood:             -2,624.375
BIC:                        5,626.290   LL-Null:                    -6,572.174
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
car                          4.1928      5.013      0.836      0.403      -5.632      14.017
transit                     -2.2526      0.494     -4.557      0.000      -3.221      -1.284
active                      31.2470   9.91e+12   3.15e-12      1.000   -1.94e+13    1.94e+13
ASC Car                      3.3182      0.152     21.798      0.000       3.020       3.617
ASC Drive_Transit           -0.0732      0.039     -1.877      0.061      -0.150       0.003
ASC Bike                     0.9199      0.219      4.206      0.000       0.491       1.349
Car TT                      -0.0248      0.003     -7.493      0.000      -0.031      -0.018
Transit TT                  -0.0046      0.002     -2.395      0.017      -0.008      -0.001
Bike TT                     -0.0326      0.003    -10.911      0.000      -0.038      -0.027
Walk TT                     -0.0156      0.002     -7.077      0.000      -0.020      -0.011
Total Jobs_30_CAR         2.131e-06   8.36e-07      2.548      0.011    4.92e-07    3.77e-06
Total Jobs_30_WALK        9.798e-06   2.91e-06      3.367      0.001    4.09e-06    1.55e-05
Sum Res Units_30_CAR     -2.039e-05   2.13e-06     -9.560      0.000   -2.46e-05   -1.62e-05
Sum Res Units_30_shared   -1.42e-05   3.73e-06     -3.810      0.000   -2.15e-05   -6.89e-06
Sum Persons_30_CAR        6.381e-06   9.68e-07      6.593      0.000    4.48e-06    8.28e-06
Sum Persons_30_shared     5.051e-06   1.72e-06      2.941      0.003    1.69e-06    8.42e-06
TOD_3to6_DrTran             -0.1530      0.130     -1.181      0.238      -0.407       0.101
sector_mfg_DrTran           -0.0874      0.096     -0.909      0.363      -0.276       0.101
finance_WTran                0.9884      0.181      5.447      0.000       0.633       1.344
finance_DrTran               0.9866      0.180      5.489      0.000       0.634       1.339
scitech_DrTran               0.0363      0.035      1.042      0.297      -0.032       0.105
sector_edu_serv_bike         0.9175      0.233      3.937      0.000       0.461       1.374
sector_edu_serv_walk         1.0350      0.254      4.079      0.000       0.538       1.532
sector_healthcare_DrTran    -0.0337      0.043     -0.781      0.435      -0.118       0.051
sector_gov_WTran             0.0236      0.043      0.548      0.583      -0.061       0.108
age_16less25_shared          0.5387      0.267      2.018      0.044       0.016       1.062
age_25less40_WTran           0.0253      0.031      0.809      0.418      -0.036       0.087
age_25less40_bike            0.4445      0.235      1.891      0.059      -0.016       0.905
age_50less60_shared         -0.5111      0.197     -2.598      0.009      -0.897      -0.125
female_shared                0.7823      0.152      5.138      0.000       0.484       1.081
female_bike                 -1.4555      0.274     -5.313      0.000      -1.992      -0.919
hh_inc_less75k_shared        0.2137      0.171      1.249      0.211      -0.12

In [99]:
validate = pd.read_csv('/home/emma/activitysynth/activitysynth/data/HWtrips_validate_042519.csv')

In [100]:
validate['DRIVE'] = 1
validate['SHARED'] = 1
validate['WALK_TRANSIT'] = 1
validate['DRIVE_TRANSIT'] = 1
validate['BIKE'] = 1
validate['WALK'] = 1

In [102]:
validate["custom_id"] = np.arange(validate.shape[0], dtype=int) + 1

In [103]:
# Perform the desired conversion
validate = pl.convert_wide_to_long(validate, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=new_alt_id)

# Look at the first 9 rows of the long-format dataframe
validate.head(9).T

/home/emma/anaconda/envs/ual-model/lib/python3.6/site-packages/pylogit/choice_tools.py:453: UserWarning: Note, there are 151 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 37 variables.
  msg_2 + msg_3.format(num_vars_accounted_for))


,0,1,2,3,4,5,6,7,8
custom_id,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
mode_id,0.000000,1.000000,2.000000,3.000000,5.000000,6.000000,0.000000,1.000000,2.000000
modeHW,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
TOD_3to6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_mfg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
finance,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
scitech,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sector_edu_serv,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
sector_healthcare,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
sector_gov,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [104]:
prediction_array = example_nested.predict(validate)

In [105]:
prediction_array

array([0.85815185, 0.07247097, 0.03006816, ..., 0.00954862, 0.03839138,
       0.06635421])

In [107]:
# total_weights = np.sum(data_wide.loc[:9999, 'weights'])
# weights = new_data["weights"] / total_weights

alternative_names = ["drive", "shared ride", 
                     "walk transit", "drive transit",
                    "bike","walk"]
num_alternatives = validate["mode_id"].unique().size
market_shares = np.empty(num_alternatives)
for i in range(1, 7):
    filter_condition = (validate["mode_id"] == i).values
    num_obs_in_condition = filter_condition.sum()
#     current_weights = weights[filter_condition]
#     assert current_weights.size == num_obs_in_condition
    current_alternative_share =\
        prediction_array[filter_condition].dot(current_weights)
    market_shares[i - 1] = current_alternative_share
    
pd.Series(market_shares, index=alternative_names)

TypeError: Required argument 'b' (pos 1) not found